In [4]:
# Step 1: Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [5]:
! pip3 install ucimlrepo


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [6]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets 

In [7]:
# Ensure y is a Series and rename it to 'Class'
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]  # Extract the first column if y is a DataFrame
y = y.rename("Class")  # Rename the target column

# Combine X and y into a single DataFrame
df = pd.concat([X, y], axis=1)

In [8]:
# Display the first few rows of the combined DataFrame
df.head()

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses,Class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2


In [9]:
# Load dataset
#df = pd.read_csv("breast_cancer.csv")

In [10]:
#df.head()

In [11]:
# Step 3: Preprocess the dataset
# Replace missing values (marked as '?') with NaN and drop rows with missing values
df.replace('?', pd.NA, inplace=True)
df.dropna(inplace=True)

In [12]:
# Check for missing values
print(df.isnull().sum())

Clump_thickness                0
Uniformity_of_cell_size        0
Uniformity_of_cell_shape       0
Marginal_adhesion              0
Single_epithelial_cell_size    0
Bare_nuclei                    0
Bland_chromatin                0
Normal_nucleoli                0
Mitoses                        0
Class                          0
dtype: int64


In [13]:
# Convert 'Bare Nuclei' column to numeric
df['Bare_nuclei'] = pd.to_numeric(df['Bare_nuclei'], errors='coerce')

In [14]:
# Map the 'Class' column: 2 (benign) -> 0, 4 (malignant) -> 1
df['Class'] = df['Class'].map({2: 0, 4: 1})

In [15]:
df.head(20)

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses,Class
0,5,1,1,1,2,1.0,3,1,1,0
1,5,4,4,5,7,10.0,3,2,1,0
2,3,1,1,1,2,2.0,3,1,1,0
3,6,8,8,1,3,4.0,3,7,1,0
4,4,1,1,3,2,1.0,3,1,1,0
5,8,10,10,8,7,10.0,9,7,1,1
6,1,1,1,1,2,10.0,3,1,1,0
7,2,1,2,1,2,1.0,3,1,1,0
8,2,1,1,1,2,1.0,1,1,5,0
9,4,2,1,1,2,1.0,2,1,1,0


In [16]:
# Step 4: Define features (X) and target variable (y)
X = df.drop(columns=['Class'])  # All features except the target
y = df['Class']  # Target variable (0: benign, 1: malignant)

In [17]:
# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [18]:
# Step 6: Initialize the decision tree classifier
clf = DecisionTreeClassifier(max_depth=3, random_state=42)

In [19]:
# Step 7: Train the model on the training set
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=42)

In [20]:
# Step 8: Evaluate the model on the testing set
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Decision Tree Classifier: {accuracy * 100:.2f}%")

Accuracy of the Decision Tree Classifier: 95.12%


In [21]:
# Step 9: Visualize the Decision Tree with Higher Resolution
plt.figure(figsize=(15, 10), dpi=1200)  # Larger figure size and higher resolution
plot_tree(
    clf,  # The trained classifier
    feature_names=X.columns,  # Feature names from the dataset
    class_names=["Benign", "Malignant"],  # Class names from the dataset
    filled=True,  # Use colors to represent classes
    fontsize=10  # Font size for better readability
)
plt.title("Decision Tree for Breast Cancer Classification", fontsize=16)
plt.show()

In [22]:
# Step 10: Display the tree structure in text format
tree_rules = export_text(clf, feature_names=list(X.columns))
print("Decision Tree Rules:")
print(tree_rules)

Decision Tree Rules:
|--- Uniformity_of_cell_size <= 3.50
|   |--- Bare_nuclei <= 5.50
|   |   |--- Normal_nucleoli <= 3.50
|   |   |   |--- class: 0
|   |   |--- Normal_nucleoli >  3.50
|   |   |   |--- class: 1
|   |--- Bare_nuclei >  5.50
|   |   |--- Clump_thickness <= 2.00
|   |   |   |--- class: 0
|   |   |--- Clump_thickness >  2.00
|   |   |   |--- class: 1
|--- Uniformity_of_cell_size >  3.50
|   |--- Uniformity_of_cell_size <= 4.50
|   |   |--- Bare_nuclei <= 7.50
|   |   |   |--- class: 0
|   |   |--- Bare_nuclei >  7.50
|   |   |   |--- class: 1
|   |--- Uniformity_of_cell_size >  4.50
|   |   |--- Marginal_adhesion <= 1.50
|   |   |   |--- class: 1
|   |   |--- Marginal_adhesion >  1.50
|   |   |   |--- class: 1



# Decision Tree Explanation

This document explains how the decision tree works, which variables were used, and the rules applied at each decision point.

---

## Overview

The decision tree is a classification model that predicts a class label (`class: 0` or `class: 1`) based on several variables. At each step, the tree makes decisions by applying thresholds to variables and branching accordingly.

---

## Variables and Decision Rules

### 1. **Root Node: Uniformity_of_cell_size**
- The root node splits the data based on **Uniformity_of_cell_size**:
  - If `Uniformity_of_cell_size <= 3.50`, follow the left branch.
  - If `Uniformity_of_cell_size > 3.50`, follow the right branch.

### 2. **Left Branch (Uniformity_of_cell_size <= 3.50)**
- The next decision depends on **Bare_nuclei**:
  - **If Bare_nuclei <= 5.50**:
    - Check **Normall_nucleoli**:
      - If `Normall_nucleoli <= 3.50`: Predict **class: 0**.
      - If `Normall_nucleoli > 3.50`: Predict **class: 1**.
  - **If Bare_nuclei > 5.50**:
    - Check **Clump_thickness**:
      - If `Clump_thickness <= 2.00`: Predict **class: 0**.
      - If `Clump_thickness > 2.00`: Predict **class: 1**.

### 3. **Right Branch (Uniformity_of_cell_size > 3.50)**
- The next decision depends on whether **Uniformity_of_cell_size** is between 3.50 and 4.50:
  - **If Uniformity_of_cell_size <= 4.50**:
    - Check **Bare_nuclei**:
      - If `Bare_nuclei <= 7.50`: Predict **class: 0**.
      - If `Bare_nuclei > 7.50`: Predict **class: 1**.
  - **If Uniformity_of_cell_size > 4.50**:
    - Check **Marginal_adhesion**:
      - If `Marginal_adhesion <= 1.50`: Predict **class: 1**.
      - If `Marginal_adhesion > 1.50`: Predict **class: 1**.

---

## Summary of Decision Process

1. **Variable Selection**:
   - The tree begins by splitting on **Uniformity_of_cell_size**, indicating it has the highest predictive power.
   - Subsequent splits involve **Bare_nuclei**, **Normall_nucleoli**, **Clump_thickness**, and **Marginal_adhesion**.

2. **Class Assignment**:
   - Each leaf node represents a terminal decision where a class is assigned based on the rules applied.

3. **Predictions**:
   - **Class 0**: Typically corresponds to one category (e.g., "benign").
   - **Class 1**: Typically corresponds to the other category (e.g., "malignant").

---

## How the Tree Works
The decision tree works by:
- Splitting the dataset at each node using threshold rules.
- Branching left or right based on whether the condition is met.
- Assigning a class at the leaf nodes, based on the subset of data reaching that point.

Each decision refines the predictions by narrowing down possibilities, ensuring the most accurate classification.

---


##### The End